In [60]:
from pynq import Overlay
import numpy as np
from pynq import MMIO
import os 
from pynq import allocate
import math
import pynq.lib.dma
import time
from pynq import PL
import asyncio
from pynq import Interrupt

# 导入硬件工程

In [61]:
overlay = Overlay("/home/xilinx/pynq/overlays/base/Accelerator/Accelerator_Alexnet.bit")
CNN_Control=overlay.CNN_ACCELERATOR.CNN_Control_1 
Windows_Convert=overlay.CNN_ACCELERATOR.Windows_Data_Convert_0
dma_result=overlay.axi_dma_result       
dma_picture=overlay.axi_dma_picture_window 
dma_weight=overlay.axi_dma_weight      
dma_weight_fc=overlay.axi_dma_weight_full 

CONV_DATA_SEND = dma_picture.sendchannel
CONV_WEIGHT_SEND = dma_weight.sendchannel

POOL_DATA_SEND = dma_picture.sendchannel

FC_DATA_SEND = dma_weight.sendchannel
FC_WEIGHT_SEND = dma_weight_fc.sendchannel

RESULT_RECV= dma_result.recvchannel


Interrupt_Control_address=0x4120_0000
Interrupt_Control_range=0xFFF
Interrupt_mmio=MMIO(Interrupt_Control_address,Interrupt_Control_range)
Interrupt_Data_address=0x0000
Interrupt_Enable_address=0x0128
Interrupt_GEnable_address=0x011c
Interrupt_state_address=0x0120
Interrupt_mmio.write(Interrupt_Enable_address,0x1)
Interrupt_mmio.write(Interrupt_GEnable_address,0x1)
Interrupt_mmio.write(Interrupt_Enable_address,0x1)

In [62]:
def Get_Interrupt_settings():
    CNN_Control_mmio.write(cnn_start_address  ,1)
#     while(Interrupt_mmio.read(Interrupt_state_address)==0): 
#         Interrupt_Enable_address=0x0128
#     end=time.time()
#     cost_time=round(end-start,5)
#     Interrupt_mmio.write(Interrupt_state_address,0x1)  #清除标志
#     print('-------------time cost is--------------- '+ str(cost_time)+'s')
        

# 地址定义

In [63]:
#-------------------------------------------CNN Control模块-----------------------------------------
CNN_Control_address=0x4000_0000
CNN_Control_range=0xFFF
CNN_Control_mmio=MMIO(CNN_Control_address,CNN_Control_range)

#---------------CNN寄存器地址------------------
Conv_Kernel_Size_address=0x0;
Conv_Kernel_Num_address=0x4;
Pool_Kernel_Size_address=0x8;
Pool_Kernel_Num_address=0xc;
FULLCON_Input_Size_address=0x10;
FULLCON_Output_Size_address=0x14;
select_mode_address=0x18;
cnn_start_address=0x1c;
#---------------CNN mode------------------
MODE_IDLE      =0b00000
MODE_CONV      =0b00001
MODE_RELU      =0b00010
MODE_POOL_Max  =0b00100
MODE_POOL_Mean =0b00101
MODE_FULLCON   =0b01000
MODE_CONV_RELU =0b10000

# #---------------寄存器参量------------------
# Conv_Kernel_Size   =4;
# Conv_Kernel_Num    =4;
# Pool_Kernel_Size   =4;
# Pool_Kernel_Num    =1;
# FULLCON_Input_Size =28;
# FULLCON_Output_Size=8;
# select_mode        =MODE_CONV;
# cnn_start          =0;

#-------------------------------------------滑动窗口-----------------------------------------
Windows_Data_address=0x4000_1000
Windows_Data_range=0xFFF
Windows_mmio=MMIO(Windows_Data_address,Windows_Data_range)
Data_horizontal_address=0x0;
Data_vertical_address=0x4;
Kernel_size_address=0x8;
Stride_address=0xc;
Refresh_address=0x10;

# Data_vertical  =28
# Data_horizontal=28
# Stride          =2
# Refresh         =0

In [64]:
def Sliding_window_settings(Data_h,Data_v,Kernel_size,Stride=2,Refresh=0):
    Windows_mmio.write(Kernel_size_address,Kernel_size)
    Windows_mmio.write(Data_vertical_address,Data_v)
    Windows_mmio.write(Data_horizontal_address,Data_h)
    Windows_mmio.write(Stride_address,Stride)
    Windows_mmio.write(Refresh_address,Refresh)
    
#     Kernel_size=Windows_mmio.read(Kernel_size_address)
#     Data_vertical=Windows_mmio.read(Data_vertical_address)
#     Data_horizontal=Windows_mmio.read(Data_horizontal_address)
#     Stride=Windows_mmio.read(Stride_address)
#     Refresh=Windows_mmio.read(Refresh_address)
#     print("滑窗模块 Kernel_size核心大小      =",Kernel_size)
#     print("滑窗模块 Data_vertical图像纵向长  =",Data_vertical)
#     print("滑窗模块 Data_horizontal图像横向长=",Data_horizontal)
#     print("滑窗模块 Stride步进               =",Stride)
#     print("滑窗模块 Refresh刷新              =",Refresh)
  

In [65]:
def Convolutional_settings(matrix_conv,result_buffer,mode,Kernel_Size,Kernel_Num,Stride,conv_weight_buffer,Data_h,Data_v,input_picture_buffer,Weight_refresh=1):
    #滑动窗设置
#     Conv_input_data_vertical   = Data_v 
#     Conv_input_data_horizontal = Data_h
#     Conv_Stride =Stride
#     Conv_Kernel_Size = Kernel_Size
#     Conv_Kernel_Num = Kernel_Num

    Sliding_window_settings(Data_h,Data_v,Kernel_Size,Stride,0)
    
    #CNN 模块设置
    if(mode==MODE_CONV):
        selectmode=MODE_CONV
    else :
        selectmode=MODE_CONV_RELU
    CNN_Control_mmio.write(select_mode_address        ,selectmode        )
    CNN_Control_mmio.write(Conv_Kernel_Size_address   ,Kernel_Size   )
    CNN_Control_mmio.write(Conv_Kernel_Num_address    ,Kernel_Num    )
    
    #----------------------------传递权重----------------
#     if(Weight_refresh==1):
    CONV_WEIGHT_SEND.transfer(conv_weight_buffer)
    CONV_DATA_SEND.transfer(input_picture_buffer)
    CONV_DATA_SEND.wait()
    CONV_WEIGHT_SEND.wait()     
         
    #----------------------------传递激励数据----------------

    
    #-----------------------------开始计算，并记录时间-------------------------
    
    CNN_Control_mmio.write(cnn_start_address  ,0)
    Get_Interrupt_settings()
    
    RESULT_RECV.transfer(result_buffer)
    RESULT_RECV.wait()          
    matrix_conv+= result_buffer.T 
#     print(matrix_conv.shape)
#     hang =matrix_conv.shape[1]
#     result_buffer=result_buffer>>8
#     matrix_conv=((result_buffer.T).reshape(8,hang,hang)
             
    # 创建一个掩码，只保留低8位


# 使用位与操作提取int32_array中每个元素的低8位，并赋值给uint8_array
          
    #------------------------------结果--------------------------   
#     if(selectmode==MODE_CONV  or selectmode==MODE_CONV_RELU    ):

#         CONV_hang_size=math.floor((Conv_input_data_horizontal-Conv_Kernel_Size)/Conv_Stride+1)
#         result_data_size=CONV_hang_size*CONV_hang_size 
#         result_data_size=math.ceil(result_data_size) 
#         result_buffer = allocate(shape=(result_data_size,8), dtype=np.int16)
        #-------------------------------获取计算结果-------------------------
#         RESULT_RECV.transfer(result_buffer)
#         RESULT_RECV.wait() 
        
#         matrix_conv = np.zeros((8, result_data_size), dtype=np.int8)
#         matrix_conv=result_buffer.T
#         for j in range(int(result_data_size)):
#             for h in range(Conv_Kernel_Num):
#                 conv_result_array[h][j]=result_buffer[j][h] 
    
    return matrix_conv


In [66]:
def Pooling_settings(matrix_pool,result_buffer,mode,Data_horizontal,Data_vertical,input_picture_buffer,Kernel_Size,Kernel_Num=1,Stride=2):
   #滑动窗设置
#     Pool_input_data_vertical =Data_vertical  
#     Pool_input_data_horizontal =Data_horizontal
#     Pool_Kernel_Size =Kernel_Size
#     Pool_Kernel_Num  =Kernel_Num
#     Pool_Stride      =Kernel_Size
    
    Sliding_window_settings(Data_horizontal,Data_vertical,Kernel_Size,Stride,0)
    if(mode==MODE_POOL_Max):
        selectmode=MODE_POOL_Max
    else :
        selectmode=MODE_POOL_Mean
    CNN_Control_mmio.write(select_mode_address        ,selectmode        )
    CNN_Control_mmio.write(Pool_Kernel_Size_address   ,Kernel_Size   )
#     CNN_Control_mmio.write(Pool_Kernel_Num_address    ,Pool_Kernel_Num    )
#     print ( " 卷积核心大小    ",   Pool_Kernel_Size    )
#     print ( " 卷积核心个数    ",   Pool_Kernel_Num     ) 

    #----------------------------传递激励数据----------------
    CONV_DATA_SEND.transfer(input_picture_buffer)
    CONV_DATA_SEND.wait()
    
    #-----------------------------开始计算，并记录时间-------------------------
    
    CNN_Control_mmio.write(cnn_start_address  ,0)
    Get_Interrupt_settings()
    RESULT_RECV.transfer(result_buffer)
    RESULT_RECV.wait()
#     print("尺寸",matrix_pool.shape[0])
    for j in range(matrix_pool.shape[0]):
           matrix_pool[j]=result_buffer[j*32]    
                
                
    #---------------------------结果--------------------------        
#     if(mode==MODE_POOL_Max or mode==MODE_POOL_Mean ):
# #         if(mode==MODE_POOL_Max):
# #             print ( " CNN计算模式   :POOL_Max    ")
# #         else :
# #             print ( " CNN计算模式   :POOL_Mean    ")
#         Pool_hang_size=math.floor((Pool_input_data_horizontal-Pool_Kernel_Size)/Pool_Kernel_Size+1)
#         Pool_data_size=Pool_hang_size*Pool_hang_size
#         result_data_size=Pool_data_size*16#多的空闲位置  16bit*8
#         matrix_pool = np.zeros((Pool_data_size,), dtype=np.int8)
#         result_data_size=math.ceil(result_data_size)
#         result_buffer = allocate(shape=(result_data_size,), dtype=np.int8)
# #         print("结果输出个数---------",result_data_size)
#         RESULT_RECV.transfer(result_buffer)
#         RESULT_RECV.wait()
#         for j in range(Pool_data_size):
#                matrix_pool[j]=result_buffer[j*16]      
    
    
    
    return matrix_pool

In [67]:
def Fullconnection_settings(mode,Input_Size,Output_Size,FC_weight_buffer,FC_act_buffer):
    #滑动窗设置
    global FULLCON_Input_Size   
    global FULLCON_Output_Size  
    global start
    FULLCON_Input_Size    =Input_Size  
    FULLCON_Output_Size   =Output_Size
    #CNN 模块设置
    if(mode==MODE_FULLCON):
        selectmode=MODE_FULLCON
    else :
        selectmode=MODE_FULLCON
    CNN_Control_mmio.write(select_mode_address        ,selectmode        )

    #----------------------------传递权重----------------
    FC_WEIGHT_SEND.transfer(FC_weight_buffer)
    FC_WEIGHT_SEND.wait()
    #----------------------------传递激励数据----------------
    FC_DATA_SEND.transfer(FC_act_buffer)
    FC_DATA_SEND.wait()
    
    #-----------------------------开始计算，并记录时间-------------------------
    
    CNN_Control_mmio.write(cnn_start_address  ,0)
    Get_Interrupt_settings()
    
    
    #--------------------------------------结果---------------------------   
#     print ( " CNN计算模式   :FULLCON  ")
    result_data_size=8   #多的空闲位置  16bit*8
    result_data_size=math.ceil(result_data_size) 
    result_buffer = allocate(shape=(result_data_size,), dtype=np.int16)
    #-------------------------------获取计算结果-------------------------
    RESULT_RECV.transfer(result_buffer)
    RESULT_RECV.wait() 


# 卷积输入激励数据 权重数据

In [68]:
from PIL import Image
# 读取图像并将其转换为灰度图
def read_and_convert_image(filename):
    image = Image.open(filename).convert('L')  # 'L'表示将图像转换为灰度图
    return image

# 将图像数据存储到PYNQ分配的内存空间中
def store_image_data(image,padding,buffer):
    width, height = image.size
    for y in range(height+padding*2+padding*2):
        for x in range(width+padding*2):
            # 获取像素值并存储到分配的内存空间中
            if(y<padding or y>height-padding-1 or x<padding or x>width-padding-1):
                  buffer[y*width+x]=0
            else: 
                  buffer[y*width+x] = image.getpixel((x-padding, y-padding))

In [69]:
# 读取TXT文件
def read_txt_file(filename):
    with open(filename, 'r') as file:
        data = file.readlines()
    return data

# 将文本数据转换为整数数组
def store_txt_data(text_data,buffer):
    parsed_data = []
    for line in text_data:
        # 假设每行数据以空格分隔，并且数据都是整数
        parsed_line = [int(x,16) for x in line.split()]
        parsed_data.append(parsed_line)
    
    # 获取数据的维度
    rows = len(parsed_data)
    cols = len(parsed_data[0])
    
    # 将数据复制到分配的内存空间中
    for i in range(rows):
        for j in range(cols):
             buffer[i*cols+j]=parsed_data[i][j]   
    return    

In [55]:
# 读取图像
filename = '2.png'  # 替换为您的图像文件名
image = read_and_convert_image(filename)
# 获取图像的宽度和高度
width, height = image.size
CONV1_Padding=1
CONV1_DataV=width+CONV1_Padding*2
CONV1_DataH=height+CONV1_Padding*2
Conv1_Kernel_Size=3
Conv1_Stride=1
Conv1_Whole_Kernel_Num=32
Conv1_Kernel_Num=16
Conv1_mode=MODE_CONV_RELU
Conv1_Weight_refresh=1
#------------------------------设置输入数据--------------------------
CONV1_data_size = CONV1_DataH*CONV1_DataV
CONV1_in_buffer = allocate(shape=(CONV1_data_size,), dtype=np.int8) 
store_image_data(image,CONV1_Padding,CONV1_in_buffer)    
#------------------------------设置权重---------------------------------
Conv1_weight_buffer = allocate(shape=(Conv1_Kernel_Size*Conv1_Kernel_Size*2,), dtype=np.int64)

 ## 读取TXT文件
filename = 'AlexNet_16/Alexnet_conv_weight.txt'
text_data = read_txt_file(filename)

# 存文本数据
store_txt_data(text_data,Conv1_weight_buffer)

CONV1_result_hang=math.floor((CONV1_DataH-Conv1_Kernel_Size)/Conv1_Stride+1)
Conv1_result_data_size=CONV1_result_hang*CONV1_result_hang
Conv1_result_data_size=math.ceil(Conv1_result_data_size) 
conv1_result = np.zeros((Conv1_Whole_Kernel_Num, Conv1_result_data_size), dtype=np.int8)
Conv1_result_buffer = allocate(shape=(Conv1_result_data_size,Conv1_Kernel_Num), dtype=np.int16)

for i in range(math.ceil(Conv1_Whole_Kernel_Num/Conv1_Kernel_Num)):
    start=time.time()
    Convolutional_settings(conv1_result[i*Conv1_Kernel_Num: (i+1)*Conv1_Kernel_Num,: ],Conv1_result_buffer,Conv1_mode,Conv1_Kernel_Size,Conv1_Kernel_Num,Conv1_Stride,Conv1_weight_buffer,CONV1_DataH,CONV1_DataV,CONV1_in_buffer,Conv1_Weight_refresh)
print(Conv1_Whole_Kernel_Num,Conv1_result_data_size)

32 784


# 池化层参数输入

In [131]:
Pool1_mode=MODE_POOL_Mean
#---------------------一个输入的池化大小尺寸----------------
Pool1_DataH=math.ceil(CONV1_result_hang)
Pool1_DataV=Pool1_DataH
Pool1_Kernel_Size=2
Pool1_Stride=Pool1_Kernel_Size
Pool1_data_size = Pool1_DataH*Pool1_DataV
Pool1_buffer = allocate(shape=(Pool1_data_size,), dtype=np.int8)

#----------------------------池化输出结果-----------------------------
Pool1_result_hang_size=math.floor((Pool1_DataH-Pool1_Kernel_Size)/Pool1_Stride+1)
Pool1_result_data_size=Pool1_result_hang_size*Pool1_result_hang_size
pool1_result = np.zeros((Conv1_Whole_Kernel_Num,Pool1_result_data_size), dtype=np.int8)
Pool1_result_buffer = allocate(shape=(Pool1_result_data_size*32,), dtype=np.uint8)
for j in range (Conv1_Whole_Kernel_Num):
    np.copyto(Pool1_buffer[:len(conv1_result[j])], conv1_result[j])
    start=time.time()
    pool1_result[j]=Pooling_settings(pool1_result[j],Pool1_result_buffer,Pool1_mode,Pool1_DataH,Pool1_DataV,Pool1_buffer,Pool1_Kernel_Size,1,Pool1_Stride)
    print("done",j)

-------------time cost is--------------- 0.00091s
done 0
-------------time cost is--------------- 0.00246s
done 1
-------------time cost is--------------- 0.00086s
done 2
-------------time cost is--------------- 0.00085s
done 3
-------------time cost is--------------- 0.00134s
done 4
-------------time cost is--------------- 0.00068s
done 5
-------------time cost is--------------- 0.00066s
done 6
-------------time cost is--------------- 0.00066s
done 7
-------------time cost is--------------- 0.00069s
done 8
-------------time cost is--------------- 0.00067s
done 9
-------------time cost is--------------- 0.00066s
done 10
-------------time cost is--------------- 0.00066s
done 11
-------------time cost is--------------- 0.00068s
done 12
-------------time cost is--------------- 0.00071s
done 13
-------------time cost is--------------- 0.00067s
done 14
-------------time cost is--------------- 0.00066s
done 15
-------------time cost is--------------- 0.00068s
done 16
-------------time cost i

#  第二层卷积

In [99]:
# input_picture_buffer 初始化
Conv2_Padding=1
CONV2_DataV=Pool1_result_hang_size+Conv2_Padding*2
CONV2_DataH=CONV2_DataV
Conv2_Kernel_Size=3
Conv2_Stride=1
Conv2_Whole_Kernel_Num=64
Conv2_Kernel_Num=16
Conv2_mode=MODE_CONV_RELU
Conv2_Weight_refresh=1
#------------------------------设置输入数据--------------------------
CONV2_data_size = CONV2_DataH*CONV2_DataV
CONV2_in_buffer = allocate(shape=(CONV2_data_size,), dtype=np.int8)

#-----------------------------权重数据-------------------------------
Conv2_weight_buffer1 = allocate(shape=(Conv2_Kernel_Size*Conv2_Kernel_Size*2,), dtype=np.int64)
# 读取TXT文件
filename1 ='AlexNet_16/Alexnet_conv2_weight.txt'
text_data1 = read_txt_file(filename1)
# 存文本数据
store_txt_data(text_data1,Conv2_weight_buffer1)

#-----------------------计算结果---------------------------
CONV2_result_hang=math.floor((CONV2_DataH-Conv2_Kernel_Size)/Conv2_Stride+1)
Conv2_result_data_size=CONV2_result_hang*CONV2_result_hang
Conv2_result_data_size=math.ceil(Conv2_result_data_size) 
conv2_result = np.zeros((Conv2_Whole_Kernel_Num, Conv2_result_data_size), dtype=np.int8)
Conv2_result_buffer = allocate(shape=(Conv2_result_data_size,Conv2_Kernel_Num), dtype=np.int16)
for inchannel in range(Conv1_Whole_Kernel_Num):
        for h in range(CONV2_DataH):
            for l in range(CONV2_DataV):
                if(h<Conv2_Padding or h>CONV2_DataH-Conv2_Padding-1 or l<Conv2_Padding or l>CONV2_DataV-Conv2_Padding-1):
                    CONV2_in_buffer[h*CONV2_DataV+l]=0    
                else: 
                    CONV2_in_buffer[h*CONV2_DataV+l]=pool1_result[inchannel,(h-Conv2_Padding)*Pool1_result_hang_size+l-Conv2_Padding]   #更新输入数据 
       
        for i in range(math.ceil(Conv2_Whole_Kernel_Num/Conv2_Kernel_Num)):#分几次
            #选择不同的参数权重，这里默认都是一样的参数卷积
            start=time.time()    
            conv2_result[i*Conv2_Kernel_Num: (i+1)*Conv2_Kernel_Num,: ]+=Convolutional_settings(conv2_result[i*Conv2_Kernel_Num: (i+1)*Conv2_Kernel_Num,: ],Conv2_result_buffer,Conv2_mode,Conv2_Kernel_Size,Conv2_Kernel_Num,Conv2_Stride,Conv2_weight_buffer1,CONV2_DataH,CONV2_DataV,CONV2_in_buffer,Conv2_Weight_refresh)

-------------time cost is--------------- 0.00134s


KeyboardInterrupt: 

# 第二层池化

In [80]:
Pool2_mode=MODE_POOL_Max
#---------------------一个输入的池化大小尺寸----------------
Pool2_DataH=CONV2_result_hang
Pool2_DataV=Pool2_DataH
Pool2_Kernel_Size=2
Pool2_Stride=Pool2_Kernel_Size
Pool2_data_size = Pool2_DataH*Pool2_DataV
Pool2_buffer = allocate(shape=(Pool2_data_size,), dtype=np.int8)

#----------------------------池化输出结果-----------------------------
Pool2_result_hang_size=math.floor((Pool2_DataH-Pool2_Kernel_Size)/Pool2_Kernel_Size+1)
Pool2_result_data_size=Pool2_result_hang_size*Pool2_result_hang_size
pool2_result = np.zeros((Conv2_Whole_Kernel_Num,Pool2_result_data_size), dtype=np.int8)
Pool2_result_buffer = allocate(shape=(Pool2_result_data_size*16,), dtype=np.int8)
for n in range (Conv2_Whole_Kernel_Num):
    for h in range(Pool2_DataH):
        for l in range(Pool2_DataV):
            Pool2_buffer[h*Pool2_DataH+l] =conv2_result[n][h*Pool2_DataH+l]
    start=time.time()
    pool2_result[n]=Pooling_settings(pool2_result[n],Pool2_result_buffer,Pool2_mode,Pool2_DataH,Pool2_DataV,Pool2_buffer,Pool2_Kernel_Size,1,Pool2_Stride)

-------------time cost is--------------- 0.0008s
-------------time cost is--------------- 0.00248s
-------------time cost is--------------- 0.00081s
-------------time cost is--------------- 0.00082s
-------------time cost is--------------- 0.00069s
-------------time cost is--------------- 0.00063s
-------------time cost is--------------- 0.00065s
-------------time cost is--------------- 0.00064s
-------------time cost is--------------- 0.00063s
-------------time cost is--------------- 0.00065s
-------------time cost is--------------- 0.00063s
-------------time cost is--------------- 0.00062s
-------------time cost is--------------- 0.00065s
-------------time cost is--------------- 0.00062s
-------------time cost is--------------- 0.00062s
-------------time cost is--------------- 0.00064s
-------------time cost is--------------- 0.00062s
-------------time cost is--------------- 0.00062s
-------------time cost is--------------- 0.00063s
-------------time cost is--------------- 0.00062s
-

# 第三层卷积

In [75]:
Conv3_Padding=1
CONV3_DataV=Pool2_result_hang_size+Conv3_Padding*2
CONV3_DataH=Pool2_result_hang_size+Conv3_Padding*2
Conv3_Kernel_Size=3
Conv3_Stride=1
Conv3_Whole_Kernel_Num=128
Conv3_Kernel_Num=16
Conv3_mode=MODE_CONV
Conv3_Weight_refresh=1
#------------------------------设置输入数据--------------------------
CONV3_data_size = CONV3_DataH*CONV3_DataV
CONV3_in_buffer = allocate(shape=(CONV3_data_size,), dtype=np.int8)

#-----------------------------权重数据-------------------------------
Conv3_weight_buffer = allocate(shape=(Conv3_Kernel_Size*Conv3_Kernel_Size*2,), dtype=np.int64)
# 读取TXT文件
filename1 ='AlexNet_data/Alexnet_conv3_weight.txt'
text_data1 = read_txt_file(filename1)
# 存文本数据
store_txt_data(text_data1,Conv3_weight_buffer)

#-----------------------计算结果---------------------------
CONV3_result_hang=math.floor((CONV3_DataH-Conv3_Kernel_Size)/Conv3_Stride+1)
Conv3_result_data_size=math.ceil(CONV3_result_hang*CONV3_result_hang)

conv3_result = np.zeros((Conv3_Whole_Kernel_Num, Conv3_result_data_size), dtype=np.int8)
Conv3_result_buffer = allocate(shape=(Conv3_result_data_size,Conv3_Kernel_Num), dtype=np.int16)
for inchannel in range(Conv2_Whole_Kernel_Num):
        for h in range(CONV3_DataH):
            for l in range(CONV3_DataV):
                if(h<Conv3_Padding or h>CONV3_DataH-Conv3_Padding-1 or l<Conv3_Padding or l>CONV3_DataV-Conv3_Padding-1):
                    CONV3_in_buffer[h*CONV3_DataV+l]=0    
                else: 
                    CONV3_in_buffer[h*CONV3_DataV+l]=pool2_result[inchannel,(h-Conv3_Padding)*Pool2_result_hang_size+l-Conv3_Padding]   #更新输入数据 
       
        for i in range(math.ceil(Conv3_Whole_Kernel_Num/Conv3_Kernel_Num)):#分几次
            #选择不同的参数权重，这里默认都是一样的参数卷积
            start=time.time()    
            conv3_result[i*Conv3_Kernel_Num: (i+1)*Conv3_Kernel_Num,: ]+=Convolutional_settings(conv3_result[i*Conv3_Kernel_Num: (i+1)*Conv3_Kernel_Num,: ],Conv3_result_buffer,Conv3_mode,Conv3_Kernel_Size,Conv3_Kernel_Num,Conv3_Stride,Conv3_weight_buffer,CONV3_DataH,CONV3_DataV,CONV3_in_buffer,Conv3_Weight_refresh)

-------------time cost is--------------- 0.00235s
-------------time cost is--------------- 0.00126s
-------------time cost is--------------- 0.00177s
-------------time cost is--------------- 0.00123s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00117s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00117s
-------------time cost is--------------- 0.00111s
--------

-------------time cost is--------------- 0.00295s
-------------time cost is--------------- 0.00145s
-------------time cost is--------------- 0.00125s
-------------time cost is--------------- 0.00123s
-------------time cost is--------------- 0.00135s
-------------time cost is--------------- 0.00127s
-------------time cost is--------------- 0.00123s
-------------time cost is--------------- 0.00148s
-------------time cost is--------------- 0.00271s
-------------time cost is--------------- 0.00114s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.0012s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
------

-------------time cost is--------------- 0.00333s
-------------time cost is--------------- 0.00145s
-------------time cost is--------------- 0.00124s
-------------time cost is--------------- 0.00123s
-------------time cost is--------------- 0.00135s
-------------time cost is--------------- 0.00126s
-------------time cost is--------------- 0.00123s
-------------time cost is--------------- 0.00306s
-------------time cost is--------------- 0.00117s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00123s
-------------time cost is--------------- 0.00113s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00113s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-

-------------time cost is--------------- 0.00329s
-------------time cost is--------------- 0.00187s
-------------time cost is--------------- 0.00128s
-------------time cost is--------------- 0.00124s
-------------time cost is--------------- 0.00127s
-------------time cost is--------------- 0.00132s
-------------time cost is--------------- 0.00127s
-------------time cost is--------------- 0.00139s
-------------time cost is--------------- 0.00286s
-------------time cost is--------------- 0.00115s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00119s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-----

-------------time cost is--------------- 0.00257s
-------------time cost is--------------- 0.00135s
-------------time cost is--------------- 0.00126s
-------------time cost is--------------- 0.00128s
-------------time cost is--------------- 0.00131s
-------------time cost is--------------- 0.00127s
-------------time cost is--------------- 0.00122s
-------------time cost is--------------- 0.00299s
-------------time cost is--------------- 0.00114s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00113s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00116s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00111s
----

# 第四层卷积

In [77]:
Conv4_Padding=1
CONV4_DataV=CONV3_result_hang+Conv4_Padding*2
CONV4_DataH=CONV3_result_hang+Conv4_Padding*2
Conv4_Kernel_Size=3
Conv4_Stride=1
Conv4_Whole_Kernel_Num=256
Conv4_Kernel_Num=8
Conv4_mode=MODE_CONV
Conv4_Weight_refresh=1
#------------------------------设置输入数据--------------------------
CONV4_data_size = CONV4_DataH*CONV4_DataV
CONV4_in_buffer = allocate(shape=(CONV4_data_size,), dtype=np.int8)

#-----------------------------权重数据-------------------------------
Conv4_weight_buffer = allocate(shape=(Conv4_Kernel_Size*Conv4_Kernel_Size,), dtype=np.int64)
# 读取TXT文件
filename1 ='AlexNet_data/Alexnet_conv4_weight.txt'
text_data1 = read_txt_file(filename1)
# 存文本数据
store_txt_data(text_data1,Conv4_weight_buffer)

#-----------------------计算结果---------------------------
CONV4_result_hang=math.floor((CONV4_DataH-Conv4_Kernel_Size)/Conv4_Stride+1)
Conv4_result_data_size=math.ceil(CONV4_result_hang*CONV4_result_hang)

conv4_result = np.zeros((Conv4_Whole_Kernel_Num, Conv4_result_data_size), dtype=np.int8)
Conv4_result_buffer = allocate(shape=(Conv4_result_data_size,Conv4_Kernel_Num), dtype=np.int16)
for inchannel in range(Conv2_Whole_Kernel_Num):
        for h in range(CONV4_DataH):
            for l in range(CONV4_DataV):
                if(h<Conv4_Padding or h>CONV4_DataH-Conv4_Padding-1 or l<Conv4_Padding or l>CONV4_DataV-Conv4_Padding-1):
                    CONV4_in_buffer[h*CONV4_DataV+l]=0    
                else: 
                    CONV4_in_buffer[h*CONV4_DataV+l]=conv3_result[inchannel,(h-Conv4_Padding)*CONV3_result_hang+l-Conv4_Padding]   #更新输入数据 
       
        for i in range(math.ceil(Conv4_Whole_Kernel_Num/Conv4_Kernel_Num)):#分几次
            #选择不同的参数权重，这里默认都是一样的参数卷积
            start=time.time()    
            conv4_result[i*Conv4_Kernel_Num: (i+1)*Conv4_Kernel_Num,: ]+=Convolutional_settings(conv4_result[i*Conv4_Kernel_Num: (i+1)*Conv4_Kernel_Num,: ],Conv4_result_buffer,Conv4_mode,Conv4_Kernel_Size,Conv4_Kernel_Num,Conv4_Stride,Conv4_weight_buffer,CONV4_DataH,CONV4_DataV,CONV4_in_buffer,Conv4_Weight_refresh)

-------------time cost is--------------- 0.00147s
-------------time cost is--------------- 0.00129s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00125s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00129s
-------------time cost is--------------- 0.00113s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00133s
-------------time cost is--------------- 0.00122s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00116s
----

-------------time cost is--------------- 0.0029s
-------------time cost is--------------- 0.00134s
-------------time cost is--------------- 0.00143s
-------------time cost is--------------- 0.00124s
-------------time cost is--------------- 0.00122s
-------------time cost is--------------- 0.00157s
-------------time cost is--------------- 0.0014s
-------------time cost is--------------- 0.00136s
-------------time cost is--------------- 0.00137s
-------------time cost is--------------- 0.00135s
-------------time cost is--------------- 0.00117s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00115s
-------------time cost is--------------- 0.00108s
-----

-------------time cost is--------------- 0.00283s
-------------time cost is--------------- 0.00199s
-------------time cost is--------------- 0.00128s
-------------time cost is--------------- 0.00122s
-------------time cost is--------------- 0.00127s
-------------time cost is--------------- 0.00132s
-------------time cost is--------------- 0.00123s
-------------time cost is--------------- 0.00124s
-------------time cost is--------------- 0.00127s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00109s
-

-------------time cost is--------------- 0.00326s
-------------time cost is--------------- 0.00145s
-------------time cost is--------------- 0.00125s
-------------time cost is--------------- 0.00123s
-------------time cost is--------------- 0.00327s
-------------time cost is--------------- 0.00127s
-------------time cost is--------------- 0.00124s
-------------time cost is--------------- 0.00128s
-------------time cost is--------------- 0.00115s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00116s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00116s
-------------time cost is--------------- 0.00109s
-

-------------time cost is--------------- 0.00332s
-------------time cost is--------------- 0.00145s
-------------time cost is--------------- 0.00125s
-------------time cost is--------------- 0.00123s
-------------time cost is--------------- 0.00141s
-------------time cost is--------------- 0.00128s
-------------time cost is--------------- 0.00122s
-------------time cost is--------------- 0.0013s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00115s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
---

-------------time cost is--------------- 0.00161s
-------------time cost is--------------- 0.00144s
-------------time cost is--------------- 0.00126s
-------------time cost is--------------- 0.00122s
-------------time cost is--------------- 0.00156s
-------------time cost is--------------- 0.00138s
-------------time cost is--------------- 0.00137s
-------------time cost is--------------- 0.00138s
-------------time cost is--------------- 0.0029s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00115s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00108s
-

-------------time cost is--------------- 0.00325s
-------------time cost is--------------- 0.00154s
-------------time cost is--------------- 0.00136s
-------------time cost is--------------- 0.00132s
-------------time cost is--------------- 0.00143s
-------------time cost is--------------- 0.00149s
-------------time cost is--------------- 0.00135s
-------------time cost is--------------- 0.00136s
-------------time cost is--------------- 0.00129s
-------------time cost is--------------- 0.00119s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00118s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.0011s
---

-------------time cost is--------------- 0.0032s
-------------time cost is--------------- 0.00146s
-------------time cost is--------------- 0.00124s
-------------time cost is--------------- 0.00122s
-------------time cost is--------------- 0.00137s
-------------time cost is--------------- 0.00306s
-------------time cost is--------------- 0.00124s
-------------time cost is--------------- 0.00128s
-------------time cost is--------------- 0.00114s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00118s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00111s
-

-------------time cost is--------------- 0.00159s
-------------time cost is--------------- 0.00145s
-------------time cost is--------------- 0.00125s
-------------time cost is--------------- 0.00125s
-------------time cost is--------------- 0.00136s
-------------time cost is--------------- 0.00128s
-------------time cost is--------------- 0.00124s
-------------time cost is--------------- 0.00146s
-------------time cost is--------------- 0.00284s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00117s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00115s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00108s


-------------time cost is--------------- 0.00256s
-------------time cost is--------------- 0.00141s
-------------time cost is--------------- 0.00137s
-------------time cost is--------------- 0.00134s
-------------time cost is--------------- 0.0015s
-------------time cost is--------------- 0.00138s
-------------time cost is--------------- 0.00134s
-------------time cost is--------------- 0.0014s
-------------time cost is--------------- 0.00288s
-------------time cost is--------------- 0.0012s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00107s
-------------time cost is--------------- 0.00107s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00107s
-------------time cost is--------------- 0.00107s
-------------time cost is--------------- 0.00108s
----

# 第五层卷积+Relu

In [79]:
Conv5_Padding=1
CONV5_DataV=CONV4_result_hang+Conv5_Padding*2
CONV5_DataH=CONV4_result_hang+Conv5_Padding*2
Conv5_Kernel_Size=3
Conv5_Stride=1
Conv5_Whole_Kernel_Num=256
Conv5_Kernel_Num=8
Conv5_mode=MODE_CONV_RELU
Conv5_Weight_refresh=1
#------------------------------设置输入数据--------------------------
CONV5_data_size = CONV5_DataH*CONV5_DataV
CONV5_in_buffer = allocate(shape=(CONV5_data_size,), dtype=np.int8)

#-----------------------------权重数据-------------------------------
Conv5_weight_buffer = allocate(shape=(Conv5_Kernel_Size*Conv5_Kernel_Size,), dtype=np.int64)
# 读取TXT文件
filename1 ='AlexNet_data/Alexnet_conv5_weight.txt'
text_data1 = read_txt_file(filename1)
# 存文本数据
store_txt_data(text_data1,Conv5_weight_buffer)

#-----------------------计算结果---------------------------
CONV5_result_hang=math.floor((CONV5_DataH-Conv5_Kernel_Size)/Conv5_Stride+1)
Conv5_result_data_size=math.ceil(CONV5_result_hang*CONV5_result_hang)

conv5_result = np.zeros((Conv5_Whole_Kernel_Num, Conv5_result_data_size), dtype=np.int8)
Conv5_result_buffer = allocate(shape=(Conv5_result_data_size,Conv5_Kernel_Num), dtype=np.int16)
for inchannel in range(Conv4_Whole_Kernel_Num):
        for h in range(CONV5_DataH):
            for l in range(CONV5_DataV):
                if(h<Conv5_Padding or h>CONV5_DataH-Conv5_Padding-1 or l<Conv5_Padding or l>CONV5_DataV-Conv5_Padding-1):
                    CONV5_in_buffer[h*CONV5_DataV+l]=0    
                else: 
                    CONV5_in_buffer[h*CONV5_DataV+l]=conv4_result[inchannel,(h-Conv5_Padding)*CONV4_result_hang+l-Conv5_Padding]   #更新输入数据 
       
        for i in range(math.ceil(Conv5_Whole_Kernel_Num/Conv5_Kernel_Num)):#分几次
            #选择不同的参数权重，这里默认都是一样的参数卷积
            start=time.time()    
            conv5_result[i*Conv5_Kernel_Num: (i+1)*Conv5_Kernel_Num,: ]+=Convolutional_settings(conv5_result[i*Conv5_Kernel_Num: (i+1)*Conv5_Kernel_Num,: ],Conv5_result_buffer,Conv5_mode,Conv5_Kernel_Size,Conv5_Kernel_Num,Conv5_Stride,Conv5_weight_buffer,CONV5_DataH,CONV5_DataV,CONV5_in_buffer,Conv5_Weight_refresh)

-------------time cost is--------------- 0.00233s
-------------time cost is--------------- 0.00136s
-------------time cost is--------------- 0.00181s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00121s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00117s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-----------

-------------time cost is--------------- 0.00322s
-------------time cost is--------------- 0.0014s
-------------time cost is--------------- 0.00126s
-------------time cost is--------------- 0.00127s
-------------time cost is--------------- 0.00135s
-------------time cost is--------------- 0.00139s
-------------time cost is--------------- 0.00134s
-------------time cost is--------------- 0.00142s
-------------time cost is--------------- 0.00291s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-----

-------------time cost is--------------- 0.00286s
-------------time cost is--------------- 0.00142s
-------------time cost is--------------- 0.00127s
-------------time cost is--------------- 0.00129s
-------------time cost is--------------- 0.00153s
-------------time cost is--------------- 0.00137s
-------------time cost is--------------- 0.00134s
-------------time cost is--------------- 0.00139s
-------------time cost is--------------- 0.00121s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
--------

-------------time cost is--------------- 0.00323s
-------------time cost is--------------- 0.00146s
-------------time cost is--------------- 0.00125s
-------------time cost is--------------- 0.00123s
-------------time cost is--------------- 0.00329s
-------------time cost is--------------- 0.0013s
-------------time cost is--------------- 0.00125s
-------------time cost is--------------- 0.0013s
-------------time cost is--------------- 0.00115s
-------------time cost is--------------- 0.00113s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00116s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00118s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------

-------------time cost is--------------- 0.00328s
-------------time cost is--------------- 0.00147s
-------------time cost is--------------- 0.00134s
-------------time cost is--------------- 0.00122s
-------------time cost is--------------- 0.00138s
-------------time cost is--------------- 0.00129s
-------------time cost is--------------- 0.00303s
-------------time cost is--------------- 0.00132s
-------------time cost is--------------- 0.00115s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.0012s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00118s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
------

...

-------------time cost is--------------- 0.00331s
-------------time cost is--------------- 0.00146s
-------------time cost is--------------- 0.00126s
-------------time cost is--------------- 0.00124s
-------------time cost is--------------- 0.00144s
-------------time cost is--------------- 0.0013s
-------------time cost is--------------- 0.00126s
-------------time cost is--------------- 0.00142s
-------------time cost is--------------- 0.00116s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00119s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00116s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.0011s
--

-------------time cost is--------------- 0.00287s
-------------time cost is--------------- 0.00128s
-------------time cost is--------------- 0.00145s
-------------time cost is--------------- 0.00127s
-------------time cost is--------------- 0.00126s
-------------time cost is--------------- 0.00139s
-------------time cost is--------------- 0.00129s
-------------time cost is--------------- 0.00125s
-------------time cost is--------------- 0.00147s
-------------time cost is--------------- 0.00269s
-------------time cost is--------------- 0.00114s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.00128s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00116s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00111s
-

-------------time cost is--------------- 0.00164s
-------------time cost is--------------- 0.00147s
-------------time cost is--------------- 0.00136s
-------------time cost is--------------- 0.00125s
-------------time cost is--------------- 0.00141s
-------------time cost is--------------- 0.00127s
-------------time cost is--------------- 0.00139s
-------------time cost is--------------- 0.00142s
-------------time cost is--------------- 0.00286s
-------------time cost is--------------- 0.00121s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00124s
-------------time cost is--------------- 0.00118s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.00111s
-------------time cost is--------------- 0.0011s
---

-------------time cost is--------------- 0.00298s
-------------time cost is--------------- 0.00116s
-------------time cost is--------------- 0.00248s
-------------time cost is--------------- 0.00115s
-------------time cost is--------------- 0.00112s
-------------time cost is--------------- 0.0013s
-------------time cost is--------------- 0.00184s
-------------time cost is--------------- 0.00124s
-------------time cost is--------------- 0.00121s
-------------time cost is--------------- 0.0014s
-------------time cost is--------------- 0.00166s
-------------time cost is--------------- 0.00132s
-------------time cost is--------------- 0.00131s
-------------time cost is--------------- 0.00125s
-------------time cost is--------------- 0.00277s
-------------time cost is--------------- 0.00108s
-------------time cost is--------------- 0.00106s
-------------time cost is--------------- 0.00106s
-------------time cost is--------------- 0.00104s
-------------time cost is--------------- 0.00106s
--

-------------time cost is--------------- 0.00295s
-------------time cost is--------------- 0.00191s
-------------time cost is--------------- 0.00125s
-------------time cost is--------------- 0.00123s
-------------time cost is--------------- 0.00122s
-------------time cost is--------------- 0.00153s
-------------time cost is--------------- 0.00133s
-------------time cost is--------------- 0.00128s
-------------time cost is--------------- 0.00134s
-------------time cost is--------------- 0.00119s
-------------time cost is--------------- 0.00275s
-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00106s
-------------time cost is--------------- 0.00106s
-------------time cost is--------------- 0.00105s
-------------time cost is--------------- 0.00106s
-------------time cost is--------------- 0.00106s
-------------time cost is--------------- 0.00106s
-------------time cost is--------------- 0.00105s
-------------time cost is--------------- 0.00105s


# 第三次池化

In [86]:
Pool3_mode=MODE_POOL_Max
#---------------------一个输入的池化大小尺寸----------------
Pool3_DataH=CONV5_result_hang
Pool3_DataV=CONV5_result_hang
Pool3_Kernel_Size=3
Pool3_Stride=2
Pool3_data_size = Pool3_DataH*Pool3_DataV
Pool3_buffer = allocate(shape=(Pool3_data_size,), dtype=np.int8)
print()
#----------------------------池化输出结果-----------------------------
Pool3_result_hang_size=math.floor((Pool3_DataH-Pool3_Kernel_Size)/Pool3_Stride+1)
Pool3_result_data_size=Pool3_result_hang_size*Pool3_result_hang_size
pool3_result = np.zeros((Conv5_Whole_Kernel_Num,Pool3_result_data_size), dtype=np.int8)
Pool3_result_buffer = allocate(shape=(Pool3_result_data_size*16,), dtype=np.int8)
for n in range (Conv5_Whole_Kernel_Num):
    for h in range(Pool3_DataH):
        for l in range(Pool3_DataV):
            Pool3_buffer[h*Pool3_DataV+l] =conv5_result[n][h*Pool3_DataV+l]
    start=time.time()
    pool3_result[n]=Pooling_settings(pool3_result[n],Pool3_result_buffer,Pool3_mode,Pool3_DataH,Pool3_DataV,Pool3_buffer,Pool3_Kernel_Size,1,Pool3_Stride)


-------------time cost is--------------- 0.00085s
-------------time cost is--------------- 0.00068s
-------------time cost is--------------- 0.00098s
-------------time cost is--------------- 0.00068s
-------------time cost is--------------- 0.00076s
-------------time cost is--------------- 0.00067s
-------------time cost is--------------- 0.0008s
-------------time cost is--------------- 0.00069s
-------------time cost is--------------- 0.00067s
-------------time cost is--------------- 0.00066s
-------------time cost is--------------- 0.00067s
-------------time cost is--------------- 0.00068s
-------------time cost is--------------- 0.00066s
-------------time cost is--------------- 0.00066s
-------------time cost is--------------- 0.00066s
-------------time cost is--------------- 0.00069s
-------------time cost is--------------- 0.00066s
-------------time cost is--------------- 0.0007s
-------------time cost is--------------- 0.00066s
-------------time cost is--------------- 0.00067s
-

-------------time cost is--------------- 0.00109s
-------------time cost is--------------- 0.00089s
-------------time cost is--------------- 0.0008s
-------------time cost is--------------- 0.00078s
-------------time cost is--------------- 0.00083s
-------------time cost is--------------- 0.00078s
-------------time cost is--------------- 0.00077s
-------------time cost is--------------- 0.00083s
-------------time cost is--------------- 0.00077s
-------------time cost is--------------- 0.00074s
-------------time cost is--------------- 0.00066s
-------------time cost is--------------- 0.00066s
-------------time cost is--------------- 0.00066s
-------------time cost is--------------- 0.00065s
-------------time cost is--------------- 0.00065s
-------------time cost is--------------- 0.00065s
-------------time cost is--------------- 0.0007s
-------------time cost is--------------- 0.00066s
-------------time cost is--------------- 0.00065s
-------------time cost is--------------- 0.00065s
--

In [87]:
print(Pool3_result_hang_size)

3


# 全连接第一层

In [90]:
fc_data=pool3_result.flatten()

In [92]:

FC1_Input_Size=fc_data.shape[0]
FC1_Output_Size=1024
fc1_weight = np.zeros((FC1_Output_Size,FC1_Input_Size), dtype=np.int8)
fc1_result = np.zeros((FC1_Output_Size,), dtype=np.int16)
# 读取TXT文件
filename = 'AlexNet_data/Alexnet_fc1_weight.txt'
text_data = read_txt_file(filename)
# 存文本数据
for i in range(FC1_Output_Size):
    store_txt_data(text_data,fc1_weight[i])
    for n in range(FC1_Input_Size):
        fc1_result[i]+=fc1_weight[i,n]*fc_data[n]

# 全连接第二层

In [93]:
FC2_Input_Size=fc1_result.shape[0]
FC2_Output_Size=512
fc2_weight = np.zeros((FC2_Output_Size,FC2_Input_Size), dtype=np.int8)
fc2_result = np.zeros((FC2_Output_Size,), dtype=np.int16)
# 读取TXT文件
filename = 'AlexNet_data/Alexnet_fc2_weight.txt'
text_data = read_txt_file(filename)
# 存文本数据
for i in range(FC2_Output_Size):
    store_txt_data(text_data,fc2_weight[i])
    for n in range(FC2_Input_Size):
        fc2_result[i]+=fc2_weight[i,n]*fc1_result[n]

# 全连接第三层

In [94]:

# input_weightfc_buffer 初始化  一次传入FULLCON_Output_Size个数据
FULLCON_Input_Size=FC2_Output_Size
FULLCON_Output_Size1=5
FULLCON_Output_Size2=5
FC_weight_buffer1 = allocate(shape=(FULLCON_Input_Size,), dtype=np.int64)
FC_weight_buffer2 = allocate(shape=(FULLCON_Input_Size,), dtype=np.int64)

# 读取TXT文件
filename =  'AlexNet_data/Alexnet_fc3_weight.txt'
text_data = read_txt_file(filename)
# 存文本数据
store_txt_data(text_data,FC_weight_buffer1)
# 读取TXT文件
filename =  'AlexNet_data/Alexnet_fc3_weight.txt'
text_data = read_txt_file(filename)
# 存文本数据
store_txt_data(text_data,FC_weight_buffer2)
    
FC_whole_act_data_size = FULLCON_Input_Size
FC_act_buffer = allocate(shape=(FC_whole_act_data_size,), dtype=np.int64)

FC_mode1=MODE_FULLCON
FC_mode2=MODE_FULLCON

Input_Size=FULLCON_Input_Size
FC_Output_Size1=FULLCON_Output_Size1
FC_Output_Size2=FULLCON_Output_Size2



for i in range(FC_whole_act_data_size):
    FC_act_buffer[i] = fc2_result[i]
start=time.time()
Fullconnection_settings(FC_mode1,Input_Size,FC_Output_Size1,FC_weight_buffer1,FC_act_buffer)
start=time.time()
Fullconnection_settings(FC_mode2,Input_Size,FC_Output_Size2,FC_weight_buffer2,FC_act_buffer)

-------------time cost is--------------- 0.00151s
-------------time cost is--------------- 0.00149s


# Alexnet

In [56]:
# 读取图像
filename = '2.png'  # 替换为您的图像文件名
image = read_and_convert_image(filename)
# 获取图像的宽度和高度
width, height = image.size
CONV1_Padding=1
CONV1_DataV=width+CONV1_Padding*2
CONV1_DataH=height+CONV1_Padding*2
Conv1_Kernel_Size=3
Conv1_Stride=1
Conv1_Whole_Kernel_Num=32
Conv1_Kernel_Num=16
Conv1_mode=MODE_CONV_RELU
Conv1_Weight_refresh=1
#------------------------------设置输入数据--------------------------
CONV1_data_size = CONV1_DataH*CONV1_DataV
CONV1_in_buffer = allocate(shape=(CONV1_data_size,), dtype=np.int8) 
store_image_data(image,CONV1_Padding,CONV1_in_buffer)    
#------------------------------设置权重---------------------------------
Conv1_weight_buffer = allocate(shape=(Conv1_Kernel_Size*Conv1_Kernel_Size*2,), dtype=np.int64)

 ## 读取TXT文件
filename = 'AlexNet_16/Alexnet_conv_weight.txt'
text_data = read_txt_file(filename)

# 存文本数据
store_txt_data(text_data,Conv1_weight_buffer)



CONV1_result_hang=math.floor((CONV1_DataH-Conv1_Kernel_Size)/Conv1_Stride+1)
Conv1_result_data_size=CONV1_result_hang*CONV1_result_hang
Conv1_result_data_size=math.ceil(Conv1_result_data_size) 
conv1_result = np.zeros((Conv1_Whole_Kernel_Num, Conv1_result_data_size), dtype=np.int8)
# conv1_result=allocate(shape=(Conv1_Whole_Kernel_Num, Conv1_result_data_size), dtype=np.int8)
Conv1_result_buffer = allocate(shape=(Conv1_result_data_size,Conv1_Kernel_Num), dtype=np.int16)








Pool1_mode=MODE_POOL_Max
#---------------------一个输入的池化大小尺寸----------------
Pool1_DataH=math.ceil(CONV1_result_hang)
Pool1_DataV=Pool1_DataH
Pool1_Kernel_Size=2
Pool1_Stride=Pool1_Kernel_Size
Pool1_data_size = Pool1_DataH*Pool1_DataV
Pool1_buffer = allocate(shape=(Pool1_data_size,), dtype=np.int8)

#----------------------------池化输出结果-----------------------------
Pool1_result_hang_size=math.floor((Pool1_DataH-Pool1_Kernel_Size)/Pool1_Kernel_Size+1)
Pool1_result_data_size=Pool1_result_hang_size*Pool1_result_hang_size
pool1_result = np.zeros((Conv1_Whole_Kernel_Num,Pool1_result_data_size), dtype=np.int8)
# pool1_result = allocate(shape=(Conv1_Whole_Kernel_Num,Pool1_result_data_size), dtype=np.int8)
Pool1_result_buffer = allocate(shape=(Pool1_result_data_size*32,), dtype=np.int8)





# input_picture_buffer 初始化
Conv2_Padding=1
CONV2_DataV=Pool1_result_hang_size+Conv2_Padding*2
CONV2_DataH=CONV2_DataV
Conv2_Kernel_Size=3
Conv2_Stride=1
Conv2_Whole_Kernel_Num=64
Conv2_Kernel_Num=16
Conv2_mode=MODE_CONV_RELU
Conv2_Weight_refresh=1
#------------------------------设置输入数据--------------------------
CONV2_data_size = CONV2_DataH*CONV2_DataV
CONV2_in_buffer = allocate(shape=(CONV2_DataH,CONV2_DataV), dtype=np.int8)

#-----------------------------权重数据-------------------------------
Conv2_weight_buffer1 = allocate(shape=(Conv2_Kernel_Size*Conv2_Kernel_Size*2,), dtype=np.int64)
# 读取TXT文件
filename1 ='AlexNet_16/Alexnet_conv2_weight.txt'
text_data1 = read_txt_file(filename1)
# 存文本数据
store_txt_data(text_data1,Conv2_weight_buffer1)

#-----------------------计算结果---------------------------
CONV2_result_hang=math.floor((CONV2_DataH-Conv2_Kernel_Size)/Conv2_Stride+1)
Conv2_result_data_size=CONV2_result_hang*CONV2_result_hang
Conv2_result_data_size=math.ceil(Conv2_result_data_size) 
conv2_result = np.zeros((Conv2_Whole_Kernel_Num, Conv2_result_data_size), dtype=np.int8)
Conv2_result_buffer = allocate(shape=(Conv2_result_data_size,Conv2_Kernel_Num), dtype=np.int16)








Pool2_mode=MODE_POOL_Max
#---------------------一个输入的池化大小尺寸----------------
Pool2_DataH=CONV2_result_hang
Pool2_DataV=Pool2_DataH
Pool2_Kernel_Size=2
Pool2_Stride=Pool2_Kernel_Size
Pool2_data_size = Pool2_DataH*Pool2_DataV
Pool2_buffer = allocate(shape=(Pool2_data_size,), dtype=np.int8)

#----------------------------池化输出结果-----------------------------
Pool2_result_hang_size=math.floor((Pool2_DataH-Pool2_Kernel_Size)/Pool2_Kernel_Size+1)
Pool2_result_data_size=Pool2_result_hang_size*Pool2_result_hang_size
pool2_result = np.zeros((Conv2_Whole_Kernel_Num,Pool2_result_data_size), dtype=np.int8)
Pool2_result_buffer = allocate(shape=(Pool2_result_data_size*32,), dtype=np.int8)




Conv3_Padding=1
CONV3_DataV=Pool2_result_hang_size+Conv3_Padding*2
CONV3_DataH=Pool2_result_hang_size+Conv3_Padding*2
Conv3_Kernel_Size=3
Conv3_Stride=1
Conv3_Whole_Kernel_Num=128
Conv3_Kernel_Num=16
Conv3_mode=MODE_CONV
Conv3_Weight_refresh=1
#------------------------------设置输入数据--------------------------
CONV3_data_size = CONV3_DataH*CONV3_DataV
CONV3_in_buffer = allocate(shape=(CONV3_DataH,CONV3_DataV), dtype=np.int8)

#-----------------------------权重数据-------------------------------
Conv3_weight_buffer = allocate(shape=(Conv3_Kernel_Size*Conv3_Kernel_Size*2,), dtype=np.int64)
# 读取TXT文件
filename1 ='AlexNet_16/Alexnet_conv3_weight.txt'
text_data1 = read_txt_file(filename1)
# 存文本数据
store_txt_data(text_data1,Conv3_weight_buffer)

#-----------------------计算结果---------------------------
CONV3_result_hang=math.floor((CONV3_DataH-Conv3_Kernel_Size)/Conv3_Stride+1)
Conv3_result_data_size=math.ceil(CONV3_result_hang*CONV3_result_hang)

conv3_result = np.zeros((Conv3_Whole_Kernel_Num, Conv3_result_data_size), dtype=np.int8)
Conv3_result_buffer = allocate(shape=(Conv3_result_data_size,Conv3_Kernel_Num), dtype=np.int16)









Conv4_Padding=1
CONV4_DataV=CONV3_result_hang+Conv4_Padding*2
CONV4_DataH=CONV3_result_hang+Conv4_Padding*2
Conv4_Kernel_Size=3
Conv4_Stride=1
Conv4_Whole_Kernel_Num=256
Conv4_Kernel_Num=16
Conv4_mode=MODE_CONV
Conv4_Weight_refresh=1
#------------------------------设置输入数据--------------------------
CONV4_data_size = CONV4_DataH*CONV4_DataV
CONV4_in_buffer = allocate(shape=(CONV4_DataH,CONV4_DataV), dtype=np.int8)

#-----------------------------权重数据-------------------------------
Conv4_weight_buffer = allocate(shape=(Conv4_Kernel_Size*Conv4_Kernel_Size*2,), dtype=np.int64)
# 读取TXT文件
filename1 ='AlexNet_16/Alexnet_conv4_weight.txt'
text_data1 = read_txt_file(filename1)
# 存文本数据
store_txt_data(text_data1,Conv4_weight_buffer)

#-----------------------计算结果---------------------------
CONV4_result_hang=math.floor((CONV4_DataH-Conv4_Kernel_Size)/Conv4_Stride+1)
Conv4_result_data_size=math.ceil(CONV4_result_hang*CONV4_result_hang)

conv4_result = np.zeros((Conv4_Whole_Kernel_Num, Conv4_result_data_size), dtype=np.int8)
Conv4_result_buffer = allocate(shape=(Conv4_result_data_size,Conv4_Kernel_Num), dtype=np.int16)






Conv5_Padding=1
CONV5_DataV=CONV4_result_hang+Conv5_Padding*2
CONV5_DataH=CONV4_result_hang+Conv5_Padding*2
Conv5_Kernel_Size=3
Conv5_Stride=1
Conv5_Whole_Kernel_Num=256
Conv5_Kernel_Num=16
Conv5_mode=MODE_CONV_RELU
Conv5_Weight_refresh=1
#------------------------------设置输入数据--------------------------
CONV5_data_size = CONV5_DataH*CONV5_DataV
CONV5_in_buffer = allocate(shape=(CONV5_DataH,CONV5_DataV), dtype=np.int8)

#-----------------------------权重数据-------------------------------
Conv5_weight_buffer = allocate(shape=(Conv5_Kernel_Size*Conv5_Kernel_Size*2,), dtype=np.int64)
# 读取TXT文件
filename1 ='AlexNet_16/Alexnet_conv5_weight.txt'
text_data1 = read_txt_file(filename1)
# 存文本数据
store_txt_data(text_data1,Conv5_weight_buffer)

#-----------------------计算结果---------------------------
CONV5_result_hang=math.floor((CONV5_DataH-Conv5_Kernel_Size)/Conv5_Stride+1)
Conv5_result_data_size=math.ceil(CONV5_result_hang*CONV5_result_hang)

conv5_result = np.zeros((Conv5_Whole_Kernel_Num, Conv5_result_data_size), dtype=np.int8)
Conv5_result_buffer = allocate(shape=(Conv5_result_data_size,Conv5_Kernel_Num), dtype=np.int16)




Pool3_mode=MODE_POOL_Max
#---------------------一个输入的池化大小尺寸----------------
Pool3_DataH=CONV5_result_hang
Pool3_DataV=CONV5_result_hang
Pool3_Kernel_Size=3
Pool3_Stride=2
Pool3_data_size = Pool3_DataH*Pool3_DataV
Pool3_buffer = allocate(shape=(Pool3_data_size,), dtype=np.int8)
print()
#----------------------------池化输出结果-----------------------------
Pool3_result_hang_size=math.floor((Pool3_DataH-Pool3_Kernel_Size)/Pool3_Stride+1)
Pool3_result_data_size=Pool3_result_hang_size*Pool3_result_hang_size
pool3_result = np.zeros((Conv5_Whole_Kernel_Num,Pool3_result_data_size), dtype=np.int8)
Pool3_result_buffer = allocate(shape=(Pool3_result_data_size*32,), dtype=np.int8)

In [72]:
start2=time.time()
for i in range(math.ceil(Conv1_Whole_Kernel_Num/Conv1_Kernel_Num)):
    Convolutional_settings(conv1_result[i*Conv1_Kernel_Num: (i+1)*Conv1_Kernel_Num,:],Conv1_result_buffer,Conv1_mode,Conv1_Kernel_Size,Conv1_Kernel_Num,Conv1_Stride,Conv1_weight_buffer,CONV1_DataH,CONV1_DataV,CONV1_in_buffer,Conv1_Weight_refresh)
end1=time.time()      
print("卷积RELU1",end1-start2)


for j in range (Conv1_Whole_Kernel_Num):
    np.copyto(Pool1_buffer[:len(conv1_result[j])], conv1_result[j])
    if(j==0):
        start1=time.time() 
    pool1_result[j]=Pooling_settings(pool1_result[j],Pool1_result_buffer,Pool1_mode,Pool1_DataH,Pool1_DataV, Pool1_buffer,Pool1_Kernel_Size,1,Pool1_Stride)
end1=time.time()      
print("池化1    ",end1-start1)

    
for inchannel in range(Conv1_Whole_Kernel_Num):
        buffer=np.reshape(pool1_result[inchannel], (CONV2_DataH-2*Conv2_Padding, CONV2_DataH-2*Conv2_Padding))
        padded_input = np.pad(buffer, ((Conv2_Padding, Conv2_Padding), (Conv2_Padding, Conv2_Padding)), mode='constant', constant_values=(0,0))
        np.copyto(CONV2_in_buffer, padded_input)
        if(inchannel==0):
            start1=time.time() 
        for i in range(math.ceil(Conv2_Whole_Kernel_Num/Conv2_Kernel_Num)):#分几次
            #选择不同的参数权重，这里默认都是一样的参数卷积  
            Convolutional_settings(conv2_result[i*Conv2_Kernel_Num: (i+1)*Conv2_Kernel_Num,: ],Conv2_result_buffer,Conv2_mode,Conv2_Kernel_Size,Conv2_Kernel_Num,Conv2_Stride,Conv2_weight_buffer1,CONV2_DataH,CONV2_DataV,CONV2_in_buffer,Conv2_Weight_refresh)    
end1=time.time()      
print("卷积RELU2",end1-start1)

          
            
            
for n in range (Conv2_Whole_Kernel_Num):
    np.copyto(Pool2_buffer[:len(conv2_result[n])], conv2_result[n])
    if(n==0):
        start1=time.time() 
    pool2_result[n]=Pooling_settings(pool2_result[n],Pool2_result_buffer,Pool2_mode,Pool2_DataH,Pool2_DataV,Pool2_buffer,Pool2_Kernel_Size,1,Pool2_Stride)
end1=time.time()      
print("池化2    ",end1-start1)

  
    
    
for inchannel in range(Conv2_Whole_Kernel_Num):
        buffer=np.reshape(pool2_result[inchannel], (CONV3_DataH-2*Conv3_Padding, CONV3_DataH-2*Conv3_Padding))
        np.copyto(CONV3_in_buffer[Conv3_Padding:CONV3_DataH-Conv3_Padding,Conv3_Padding:CONV3_DataV-Conv3_Padding], buffer)
        if(inchannel==0):
            start1=time.time()  
        for i in range(math.ceil(Conv3_Whole_Kernel_Num/Conv3_Kernel_Num)):#分几次
            #选择不同的参数权重，这里默认都是一样的参数卷积
            start=time.time()    
            Convolutional_settings(conv3_result[i*Conv3_Kernel_Num: (i+1)*Conv3_Kernel_Num,: ],Conv3_result_buffer,Conv3_mode,Conv3_Kernel_Size,Conv3_Kernel_Num,Conv3_Stride,Conv3_weight_buffer,CONV3_DataH,CONV3_DataV,CONV3_in_buffer,Conv3_Weight_refresh)   
end1=time.time()      
print("卷积3    ",end1-start1)




for inchannel in range(Conv3_Whole_Kernel_Num):
        buffer=np.reshape(conv3_result[inchannel], (CONV4_DataH-2*Conv4_Padding, CONV4_DataH-2*Conv4_Padding))
        np.copyto(CONV4_in_buffer[Conv4_Padding:CONV4_DataH-Conv4_Padding,Conv4_Padding:CONV4_DataV-Conv4_Padding], buffer)
        if(inchannel==0):
            start1=time.time()  
        for i in range(math.ceil(Conv4_Whole_Kernel_Num/Conv4_Kernel_Num)):#分几次
            #选择不同的参数权重，这里默认都是一样的参数卷积
            start=time.time()    
            Convolutional_settings(conv4_result[i*Conv4_Kernel_Num: (i+1)*Conv4_Kernel_Num,: ],Conv4_result_buffer,Conv4_mode,Conv4_Kernel_Size,Conv4_Kernel_Num,Conv4_Stride,Conv4_weight_buffer,CONV4_DataH,CONV4_DataV,CONV4_in_buffer,Conv4_Weight_refresh)
end1=time.time()      
print("卷积4    ",end1-start1)
 
for inchannel in range(Conv4_Whole_Kernel_Num):
        buffer=np.reshape(conv4_result[inchannel], (CONV5_DataH-2*Conv5_Padding, CONV5_DataH-2*Conv5_Padding))
        np.copyto(CONV5_in_buffer[Conv5_Padding:CONV5_DataH-Conv5_Padding,Conv5_Padding:CONV5_DataV-Conv5_Padding], buffer)
        if(inchannel==0):
            start1=time.time()  
        for i in range(math.ceil(Conv5_Whole_Kernel_Num/Conv5_Kernel_Num)):#分几次
            #选择不同的参数权重，这里默认都是一样的参数卷积   
            Convolutional_settings(conv5_result[i*Conv5_Kernel_Num: (i+1)*Conv5_Kernel_Num,: ],Conv5_result_buffer,Conv5_mode,Conv5_Kernel_Size,Conv5_Kernel_Num,Conv5_Stride,Conv5_weight_buffer,CONV5_DataH,CONV5_DataV,CONV5_in_buffer,Conv5_Weight_refresh)   
end1=time.time()      
print("卷积RELU5",end1-start1)


            
            
for n in range (Conv5_Whole_Kernel_Num):
    np.copyto(Pool3_buffer[:len(conv5_result[n])], conv5_result[n])
    if(n==0):
        start1=time.time()
    pool3_result[n]=Pooling_settings(pool3_result[n],Pool3_result_buffer,Pool3_mode,Pool3_DataH,Pool3_DataV,Pool3_buffer,Pool3_Kernel_Size,1,Pool3_Stride)
            
end1=time.time()      
print("池化3    ",end1-start1)
end2=time.time()  
print("全部    ",end2-start2)

卷积RELU1 0.007872581481933594
池化1     0.062274932861328125
卷积RELU2 0.2750382423400879
池化2     0.08728146553039551
卷积3     0.8694794178009033
卷积4     3.4437813758850098
卷积RELU5 6.9230780601501465
池化3     0.31632161140441895
全部     12.00607943534851


In [116]:
import ctypes

In [125]:
# 为DMA分配一块内存，大小为1024字节
buffer_size = 10240
dma_buffer = allocate(shape=(buffer_size,), dtype=np.uint8)
 
# 使用numpy创建一个数组，并将其数据类型转换为c_uint8
data_array = np.arange(buffer_size, dtype=np.uint8)
dma_buffer = dma_buffer#np.ones(buffer_size, dtype=np.uint8)
 


In [28]:
import numpy as np

# 创建一个32x32的数组
array_32x32 = np.random.randint(600, 3000, size=(36, 8), dtype=np.int16)

array_32x32T=array_32x32.T
# 将数组reshape为2x16x32的形状
array_32x32 = np.random.randint(0, 100, size=(8,6, 6), dtype=np.int8)
array_2x16x32 = (array_32x32T>>8).reshape(8, 6, 6)

print("32x32 array:")
print(array_32x32)
print("32x32T array:")
print(array_32x32T)
print("\nReshaped array:")
print(array_2x16x32)


32x32 array:
[[[91  1  4 33 65  5]
  [24 74 17 47  2 89]
  [69 11 52  0 25 22]
  [18 97 24 62 28 41]
  [40 92 44  3 69 92]
  [73 39 84 66  6 42]]

 [[75 25 20 85 33  1]
  [77 84 55 47 48 67]
  [64 65 55  6 88 65]
  [18  0 39 57  5 92]
  [ 8 95 10 56 83 54]
  [20 20 17  6 47 73]]

 [[59 98 30 86 72 76]
  [47 87 88  5  2  8]
  [11 78 60 38 19 81]
  [91  9 45 61 74 34]
  [85 97 76 98 76 69]
  [ 4 46 25 72 18 72]]

 [[90 52 22 13  7 17]
  [95 18 80 36 76 94]
  [ 4 75 32 13 96 16]
  [14 65 74 76 60 85]
  [ 9 87 67 48 17 87]
  [86 85 73 27 70 79]]

 [[82 65 33 88 41 94]
  [76 67 13 98 31 41]
  [84 22 49 95 43 41]
  [78 30 21 58 49 59]
  [73 74 80 55 77 91]
  [95 66 35 69 52 26]]

 [[93 22 77 84 42 67]
  [85 72 36 37 82 99]
  [ 3 77 57 48 49 71]
  [61 40 91 50 26  2]
  [96 50 79 18 78 75]
  [11 10 73 83 38 84]]

 [[52 73 39 75 57 19]
  [84 74 40 11 30 43]
  [21 98 69 29 68 80]
  [54 55 53 56 65 80]
  [50 12 69 14 82 42]
  [40  6 13 16  1 71]]

 [[63 74  3 44 40 52]
  [43 75 64 75 61  8]
  [99

In [35]:
import pynq
import numpy as np

# 假设你已经分配了两个不同位宽的数据类型的数组
# 这里使用int32和uint8作为例子
int32_array = pynq.allocate((10,), dtype=np.int32)
uint8_array = pynq.allocate((10,), dtype=np.uint8)

# 填充int32_array一些值作为例子
int32_array[:] = np.array([1, 2, 3, 4, 5,
                           6, 6, 6, 6, 6], dtype=np.int32)

# 创建一个掩码，只保留低8位
mask = np.uint32(0xFF)  # 8位全为1，其余位为0 (即 0b11111111)

# 使用位与操作提取int32_array中每个元素的低8位，并赋值给uint8_array
uint8_array[:] = (int32_array & mask).astype(np.uint8)

# 打印结果以验证
print(uint8_array[:])

[1 2 3 4 5 6 6 6 6 6]
